In [74]:
import pandas as pd

# Load the dataset
df = pd.read_csv('gdb9_G4MP2_withdata_hydrogenation_clean.csv')

# Display the first few rows of the dataframe to understand its structure
print(df.head())


  unsat_SMILE sat_SMILE     delta_H  nH2    pH2
0         C#C        CC  150.735206    2  13.42
1         C=O        CO   83.774454    1   6.29
2        CC#C       CCC  139.811813    2   9.15
3        CC=O       CCO   63.227291    1   4.38
4     CC(C)=O    CC(C)O   51.916637    1   3.36


In [75]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import rdkit.Chem.Fragments as Fragments
import rdkit.Chem as Chem
import rdkit.Chem.Crippen as Crippen
import rdkit.Chem.Lipinski as Lipinski
import rdkit.Chem.rdMolDescriptors as MolDescriptors
import rdkit.Chem.Descriptors as Descriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.decomposition import PCA
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C ,WhiteKernel as Wht,Matern as matk

In [62]:
from sklearn.model_selection import train_test_split

# Initial split: 5% for initial training, 95% as unlabeled pool
initial_data, unlabeled_pool = train_test_split(df, test_size=0.2, random_state=42)

print("Initial training data size:", initial_data.shape[0])
print("Unlabeled pool size:", unlabeled_pool.shape[0])


Initial training data size: 970
Unlabeled pool size: 18436


In [76]:

def evaluate_chem_mol(mol):
    mol_sssr = Chem.GetSSSR(mol)
    clogp    = Crippen.MolLogP(mol)
    mr       = Crippen.MolMR(mol)
    mw       = MolDescriptors.CalcExactMolWt(mol)
    tpsa    = MolDescriptors.CalcTPSA(mol)
    Chi0n    = MolDescriptors.CalcChi0n(mol)
    Chi1n    = MolDescriptors.CalcChi1n(mol)
    Chi2n    = MolDescriptors.CalcChi2n(mol)
    Chi3n    = MolDescriptors.CalcChi3n(mol)
    Chi4n    = MolDescriptors.CalcChi4n(mol)
    Chi0v    = MolDescriptors.CalcChi0v(mol)
    Chi1v    = MolDescriptors.CalcChi1v(mol)
    Chi2v    = MolDescriptors.CalcChi2v(mol)
    Chi3v    = MolDescriptors.CalcChi3v(mol)
    Chi4v    = MolDescriptors.CalcChi4v(mol)
    fracsp3  = MolDescriptors.CalcFractionCSP3(mol)
    Hall_Kier_Alpha = MolDescriptors.CalcHallKierAlpha(mol)
    Kappa1      = MolDescriptors.CalcKappa1(mol)
    Kappa2      = MolDescriptors.CalcKappa2(mol)
    Kappa3      = MolDescriptors.CalcKappa3(mol)
    LabuteASA   = MolDescriptors.CalcLabuteASA(mol)
    Number_Aliphatic_Rings = MolDescriptors.CalcNumAliphaticRings(mol)
    Number_Aromatic_Rings = MolDescriptors.CalcNumAromaticRings(mol)
    Number_Amide_Bonds = MolDescriptors.CalcNumAmideBonds(mol)
    Number_Atom_Stereocenters = MolDescriptors.CalcNumAtomStereoCenters(mol)
    Number_BridgeHead_Atoms = MolDescriptors.CalcNumBridgeheadAtoms(mol)
    Number_HBA = MolDescriptors.CalcNumHBA(mol)
    Number_HBD = MolDescriptors.CalcNumHBD(mol)
    Number_Hetero_Atoms = MolDescriptors.CalcNumHeteroatoms(mol)
    Number_Hetero_Cycles = MolDescriptors.CalcNumHeterocycles(mol)
    Number_Rings = MolDescriptors.CalcNumRings(mol)
    Number_Rotatable_Bonds = MolDescriptors.CalcNumRotatableBonds(mol)
    Number_Spiro = MolDescriptors.CalcNumSpiroAtoms(mol)
    Number_Saturated_Rings = MolDescriptors.CalcNumSaturatedRings(mol)
    Number_Heavy_Atoms = Lipinski.HeavyAtomCount(mol)
    Number_NH_OH = Lipinski.NHOHCount(mol)
    Number_N_O = Lipinski.NOCount(mol)
    Number_Valence_Electrons = Descriptors.NumValenceElectrons(mol)
    Max_Partial_Charge = Descriptors.MaxPartialCharge(mol)
    Min_Partial_Charge = Descriptors.MinPartialCharge(mol)

    return mol_sssr, clogp, mr, mw, tpsa, Chi0n, Chi1n, Chi2n, Chi3n, Chi4n, Chi0v, Chi1v, Chi2v, Chi3v, Chi4v, fracsp3, Hall_Kier_Alpha,Kappa1, Kappa2, Kappa3, LabuteASA, Number_Aliphatic_Rings, Number_Aromatic_Rings, Number_Amide_Bonds, Number_Atom_Stereocenters, Number_BridgeHead_Atoms, Number_HBA, Number_HBD, Number_Hetero_Atoms, Number_Hetero_Cycles, Number_Rings, Number_Rotatable_Bonds, Number_Spiro, Number_Saturated_Rings, Number_Heavy_Atoms, Number_NH_OH, Number_N_O, Number_Valence_Electrons, Max_Partial_Charge, Min_Partial_Charge


In [77]:

def compute_descriptors_for_dataframe(df, smiles_column):
    # Initialize lists to hold descriptors
    descriptors = []

    # Loop through SMILES strings in the dataframe
    for smiles in df[smiles_column]:
        mol = Chem.MolFromSmiles(smiles)
        if mol:  # check if mol conversion is successful
            descriptors.append(evaluate_chem_mol(mol))
        else:
            # Append None or NaN for molecules that fail conversion
            descriptors.append([None]*40)  # 40 being the number of descriptors

    # Convert list of descriptors to DataFrame
    desc_df = pd.DataFrame(descriptors, columns=[
        'mol_sssr', 'clogp', 'mr', 'mw', 'tpsa', 'Chi0n', 'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 
        'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v', 'Chi4v', 'fracsp3', 'Hall_Kier_Alpha','Kappa1', 
        'Kappa2', 'Kappa3', 'LabuteASA', 'Number_Aliphatic_Rings', 'Number_Aromatic_Rings', 
        'Number_Amide_Bonds', 'Number_Atom_Stereocenters', 'Number_BridgeHead_Atoms', 'Number_HBA', 
        'Number_HBD', 'Number_Hetero_Atoms', 'Number_Hetero_Cycles', 'Number_Rings', 'Number_Rotatable_Bonds', 
        'Number_Spiro', 'Number_Saturated_Rings', 'Number_Heavy_Atoms', 'Number_NH_OH', 'Number_N_O', 
        'Number_Valence_Electrons', 'Max_Partial_Charge', 'Min_Partial_Charge'
    ])
    
    return pd.concat([df, desc_df], axis=1)

# Compute descriptors for the entire dataset
df = compute_descriptors_for_dataframe(df, 'unsat_SMILE')


In [95]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

# Split data into initial training set and unlabeled pool
initial_idx = np.random.choice(range(len(df)), size=500, replace=False)
pool_idx = [i for i in range(len(df)) if i not in initial_idx]

X = df.iloc[:, 5:].values  # Assuming descriptors start from the 5th column
y = df['delta_H'].values

X_initial = X[initial_idx]
y_initial = y[initial_idx]
X_pool = X[pool_idx]
y_pool = y[pool_idx]

# Initialize Gaussian Process
kernel = DotProduct() + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, random_state=1)

# Train on the initial data
gp.fit(X_initial, y_initial)

# Make predictions on the pool
predictions, std_dev = gp.predict(X_pool, return_std=True)

# Let's say we want to add 100 least confident points to our training data
uncertainty_idx = np.argsort(std_dev)[-100:]

# Append the uncertain points to the initial data
X_initial = np.vstack([X_initial, X_pool[uncertainty_idx]])
y_initial = np.hstack([y_initial, y_pool[uncertainty_idx]])

# Remove the uncertain points from the pool
X_pool = np.delete(X_pool, uncertainty_idx, axis=0)
y_pool = np.delete(y_pool, uncertainty_idx)


In [96]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Before active learning loop, split out a small test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Modify your existing loop to also record performance on the test set
mse_test = []

n_iterations = 50  # Existing value
n_to_add_each_iter = 400  # Existing value

for i in range(n_iterations):
    # Existing code...
    gp.fit(X_initial, y_initial)
    predictions, std_dev = gp.predict(X_pool, return_std=True)
    uncertainty_idx = np.argsort(std_dev)[-n_to_add_each_iter:]
    X_initial = np.vstack([X_initial, X_pool[uncertainty_idx]])
    y_initial = np.hstack([y_initial, y_pool[uncertainty_idx]])
    X_pool = np.delete(X_pool, uncertainty_idx, axis=0)
    y_pool = np.delete(y_pool, uncertainty_idx)

    # New code: Record the model's performance on the test set
    y_pred_test = gp.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_test)
    mse_test.append(mse)

    print(f"Iteration {i+1}/{n_iterations} complete. Training set size: {len(y_initial)} | Test MSE: {mse:.4f}")

# After the loop, visualize the model's performance over iterations
plt.plot(range(n_iterations), mse_test, '-o')
plt.xlabel("Iteration")
plt.ylabel("Mean Squared Error on Test Set")
plt.title("Model Performance Over Iterations")
plt.show()


Iteration 1/50 complete. Training set size: 1000 | Test MSE: 967.2802
Iteration 2/50 complete. Training set size: 1400 | Test MSE: 956.0414
Iteration 3/50 complete. Training set size: 1800 | Test MSE: 920.9549
Iteration 4/50 complete. Training set size: 2200 | Test MSE: 898.9222
Iteration 5/50 complete. Training set size: 2600 | Test MSE: 859.9096
Iteration 6/50 complete. Training set size: 3000 | Test MSE: 845.8091
Iteration 7/50 complete. Training set size: 3400 | Test MSE: 821.0274
Iteration 8/50 complete. Training set size: 3800 | Test MSE: 798.2690
Iteration 9/50 complete. Training set size: 4200 | Test MSE: 782.3044
Iteration 10/50 complete. Training set size: 4600 | Test MSE: 768.7704
Iteration 11/50 complete. Training set size: 5000 | Test MSE: 754.6201
Iteration 12/50 complete. Training set size: 5400 | Test MSE: 740.5028
Iteration 13/50 complete. Training set size: 5800 | Test MSE: 731.4334
Iteration 14/50 complete. Training set size: 6200 | Test MSE: 719.1032
Iteration 15/50

KeyboardInterrupt: 

In [71]:
'''
n_iterations = 10  # You can change this to any desired number
n_to_add_each_iter = 100  # Number of points to add from the pool to training set in each iteration

for i in range(n_iterations):
    # Train Gaussian Process on current training data
    gp.fit(X_initial, y_initial)

    # Predict on the unlabeled pool
    predictions, std_dev = gp.predict(X_pool, return_std=True)

    # Select points where the model is least confident (highest standard deviation)
    uncertainty_idx = np.argsort(std_dev)[-n_to_add_each_iter:]

    # Append the uncertain points to the training data
    X_initial = np.vstack([X_initial, X_pool[uncertainty_idx]])
    y_initial = np.hstack([y_initial, y_pool[uncertainty_idx]])

    # Remove the uncertain points from the pool
    X_pool = np.delete(X_pool, uncertainty_idx, axis=0)
    y_pool = np.delete(y_pool, uncertainty_idx)

    # Optionally, print progress
    print(f"Iteration {i+1}/{n_iterations} complete. Training set size: {len(y_initial)}")

print("Active learning process complete!")
'''

'\nn_iterations = 10  # You can change this to any desired number\nn_to_add_each_iter = 100  # Number of points to add from the pool to training set in each iteration\n\nfor i in range(n_iterations):\n    # Train Gaussian Process on current training data\n    gp.fit(X_initial, y_initial)\n\n    # Predict on the unlabeled pool\n    predictions, std_dev = gp.predict(X_pool, return_std=True)\n\n    # Select points where the model is least confident (highest standard deviation)\n    uncertainty_idx = np.argsort(std_dev)[-n_to_add_each_iter:]\n\n    # Append the uncertain points to the training data\n    X_initial = np.vstack([X_initial, X_pool[uncertainty_idx]])\n    y_initial = np.hstack([y_initial, y_pool[uncertainty_idx]])\n\n    # Remove the uncertain points from the pool\n    X_pool = np.delete(X_pool, uncertainty_idx, axis=0)\n    y_pool = np.delete(y_pool, uncertainty_idx)\n\n    # Optionally, print progress\n    print(f"Iteration {i+1}/{n_iterations} complete. Training set size: {l

In [89]:
# Cc1cc2ccccc2o1

new_molecule_smiles = "c1cnc2ccc(C3CC3)cc2c1"  # Replace with your SMILES string

# Convert to RDKit mol object
new_mol = Chem.MolFromSmiles(new_molecule_smiles)

# Check if the conversion is successful
if new_mol:
    new_molecule_descriptors = evaluate_chem_mol(new_mol)
else:
    raise ValueError("Invalid SMILES string or molecule conversion failed.")

# Convert descriptors to a numpy array and reshape for single sample prediction
new_molecule_descriptors = np.array(new_molecule_descriptors).reshape(1, -1)

# Splitting the data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)

# List of descriptor columns (you've defined these when you computed descriptors)
descriptor_cols = [
    'mol_sssr', 'clogp', 'mr', 'mw', 'tpsa', 'Chi0n', 'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 
    'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v', 'Chi4v', 'fracsp3', 'Hall_Kier_Alpha','Kappa1', 
    'Kappa2', 'Kappa3', 'LabuteASA', 'Number_Aliphatic_Rings', 'Number_Aromatic_Rings', 
    'Number_Amide_Bonds', 'Number_Atom_Stereocenters', 'Number_BridgeHead_Atoms', 'Number_HBA', 
    'Number_HBD', 'Number_Hetero_Atoms', 'Number_Hetero_Cycles', 'Number_Rings', 'Number_Rotatable_Bonds', 
    'Number_Spiro', 'Number_Saturated_Rings', 'Number_Heavy_Atoms', 'Number_NH_OH', 'Number_N_O', 
    'Number_Valence_Electrons', 'Max_Partial_Charge', 'Min_Partial_Charge'
]

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data descriptors
scaler.fit(train_data[descriptor_cols])

new_molecule_descriptors_scaled = scaler.transform(new_molecule_descriptors)


predicted_delta_H = gp.predict(new_molecule_descriptors_scaled)
print(f"Predicted Hydrogenation Enthalpy (kJ/mol H2) for the new molecule: {predicted_delta_H[0]}")


Predicted Hydrogenation Enthalpy (kJ/mol H2) for the new molecule: 45.67651955256494


/Users/hassan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
